In [19]:
"""
References:
https://arxiv.org/pdf/1512.03385.pdf
https://arxiv.org/pdf/1604.04112v4.pdf
https://github.com/yu4u/cutout-random-erasing
"""
import numpy as np
from tensorflow.keras.layers import Add, Dense, Conv2D, BatchNormalization
from tensorflow.keras.layers import Activation, AveragePooling2D, Input, Flatten
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model

In [20]:
batch_size = 128
epochs = 180
n_classes = 10
learning_rate = 0.1

In [21]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)

In [22]:
class Resnet:
    def __init__(self, size=44, stacks=3, starting_filter=16):
        self.size = size
        self.stacks = stacks
        self.starting_filter = starting_filter
        self.residual_blocks = (size - 2) // 6
        
    def get_model(self, input_shape=(32, 32, 3), n_classes=10):
        n_filters = self.starting_filter

        inputs = Input(shape=input_shape)
        network = self.layer(inputs, n_filters)
        network = self.stack(network, n_filters, True)

        for _ in range(self.stacks - 1):
            n_filters *= 2
            network = self.stack(network, n_filters)

        network = Activation('elu')(network)
        network = AveragePooling2D(pool_size=network.shape[1])(network)
        network = Flatten()(network)
        outputs = Dense(n_classes, activation='softmax', 
                        kernel_initializer='he_normal')(network)

        model = Model(inputs=inputs, outputs=outputs)

        return model
    
    def stack(self, inputs, n_filters, first_stack=False):
        stack = inputs

        if first_stack:
            stack = self.identity_block(stack, n_filters)
        else:
            stack = self.convolution_block(stack, n_filters)

        for _ in range(self.residual_blocks - 1):
            stack = self.identity_block(stack, n_filters)

        return stack
    
    def identity_block(self, inputs, n_filters):
        shortcut = inputs

        block = self.layer(inputs, n_filters, normalize_batch=False)
        block = self.layer(block, n_filters, activation=None)

        block = Add()([shortcut, block])

        return block

    def convolution_block(self, inputs, n_filters, strides=2):
        shortcut = inputs

        block = self.layer(inputs, n_filters, strides=strides,
                           normalize_batch=False)
        block = self.layer(block, n_filters, activation=None)

        shortcut = self.layer(shortcut, n_filters,
                              kernel_size=1, strides=strides,
                              activation=None)

        block = Add()([shortcut, block])

        return block
    
    def layer(self, inputs, n_filters, kernel_size=3,
              strides=1, activation='elu', normalize_batch=True):
    
        convolution = Conv2D(n_filters, kernel_size=kernel_size,
                             strides=strides, padding='same',
                             kernel_initializer="he_normal",
                             kernel_regularizer=l2(1e-4))

        x = convolution(inputs)

        if normalize_batch:
            x = BatchNormalization()(x)

        if activation is not None:
            x = Activation(activation)(x)

        return x
    
def learning_rate_schedule(epoch):
    new_learning_rate = learning_rate

    if epoch <= 100:
        pass
    elif epoch > 100 and epoch <= 150:
        new_learning_rate = learning_rate * 0.1
    else:
        new_learning_rate = learning_rate * 0.01
        
    print('Learning rate:', new_learning_rate)
    
    return new_learning_rate

def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        if input_img.ndim == 3:
            img_h, img_w, img_c = input_img.shape
        elif input_img.ndim == 2:
            img_h, img_w = input_img.shape

        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            if input_img.ndim == 3:
                c = np.random.uniform(v_l, v_h, (h, w, img_c))
            if input_img.ndim == 2:
                c = np.random.uniform(v_l, v_h, (h, w))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w] = c

        return input_img

    return eraser

In [23]:
resnet = Resnet()

model = resnet.get_model()

optimizer = SGD(learning_rate=learning_rate, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_135 (Conv2D)             (None, 32, 32, 16)   448         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_72 (BatchNo (None, 32, 32, 16)   64          conv2d_135[0][0]                 
__________________________________________________________________________________________________
activation_69 (Activation)      (None, 32, 32, 16)   0           batch_normalization_72[0][0]     
____________________________________________________________________________________________

In [24]:
lr_scheduler = LearningRateScheduler(learning_rate_schedule)
callbacks = [lr_scheduler]

datagen = ImageDataGenerator(width_shift_range=4,
                             height_shift_range=4,
                             horizontal_flip=True,
                             preprocessing_function=get_random_eraser(p=1, pixel_level=True))
datagen.fit(x_train)

model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
          validation_data=(x_test, y_test),
          epochs=epochs, workers=4,
          callbacks=callbacks)

model.save('model.h5')

Epoch 1/180
Learning rate: 0.1
391/391 [==============================] - 30s 67ms/step - loss: 1.9564 - accuracy: 0.4099 - val_loss: 1.9104 - val_accuracy: 0.4790
Epoch 2/180
Learning rate: 0.1
391/391 [==============================] - 25s 63ms/step - loss: 1.5425 - accuracy: 0.5566 - val_loss: 1.6927 - val_accuracy: 0.5715
Epoch 3/180
Learning rate: 0.1
391/391 [==============================] - 25s 63ms/step - loss: 1.3299 - accuracy: 0.6321 - val_loss: 1.1486 - val_accuracy: 0.7031
Epoch 4/180
Learning rate: 0.1
391/391 [==============================] - 25s 63ms/step - loss: 1.1810 - accuracy: 0.6832 - val_loss: 1.0400 - val_accuracy: 0.7375
Epoch 5/180
Learning rate: 0.1
391/391 [==============================] - 25s 63ms/step - loss: 1.0844 - accuracy: 0.7140 - val_loss: 1.0425 - val_accuracy: 0.7334
Epoch 6/180
Learning rate: 0.1
391/391 [==============================] - 25s 63ms/step - loss: 1.0205 - accuracy: 0.7331 - val_loss: 1.2108 - val_accuracy: 0.6837
Epoch 7/180
Lear

391/391 [==============================] - 25s 63ms/step - loss: 0.6366 - accuracy: 0.8710 - val_loss: 0.6601 - val_accuracy: 0.8731
Epoch 51/180
Learning rate: 0.1
391/391 [==============================] - 24s 62ms/step - loss: 0.6467 - accuracy: 0.8684 - val_loss: 0.6894 - val_accuracy: 0.8594
Epoch 52/180
Learning rate: 0.1
391/391 [==============================] - 25s 63ms/step - loss: 0.6483 - accuracy: 0.8676 - val_loss: 0.7589 - val_accuracy: 0.8469
Epoch 53/180
Learning rate: 0.1
391/391 [==============================] - 25s 63ms/step - loss: 0.6440 - accuracy: 0.8704 - val_loss: 0.6871 - val_accuracy: 0.8641
Epoch 54/180
Learning rate: 0.1
391/391 [==============================] - 25s 63ms/step - loss: 0.6443 - accuracy: 0.8710 - val_loss: 0.6902 - val_accuracy: 0.8635
Epoch 55/180
Learning rate: 0.1
391/391 [==============================] - 25s 63ms/step - loss: 0.6511 - accuracy: 0.8683 - val_loss: 0.6677 - val_accuracy: 0.8681
Epoch 56/180
Learning rate: 0.1
391/391 [=

Epoch 100/180
Learning rate: 0.1
391/391 [==============================] - 24s 62ms/step - loss: 0.6166 - accuracy: 0.8845 - val_loss: 0.9341 - val_accuracy: 0.7996
Epoch 101/180
Learning rate: 0.1
391/391 [==============================] - 24s 62ms/step - loss: 0.6229 - accuracy: 0.8817 - val_loss: 0.7159 - val_accuracy: 0.8614
Epoch 102/180
Learning rate: 0.010000000000000002
391/391 [==============================] - 25s 63ms/step - loss: 0.5351 - accuracy: 0.9128 - val_loss: 0.4985 - val_accuracy: 0.9264
Epoch 103/180
Learning rate: 0.010000000000000002
391/391 [==============================] - 25s 63ms/step - loss: 0.4785 - accuracy: 0.9307 - val_loss: 0.4798 - val_accuracy: 0.9316
Epoch 104/180
Learning rate: 0.010000000000000002
391/391 [==============================] - 26s 65ms/step - loss: 0.4589 - accuracy: 0.9376 - val_loss: 0.4685 - val_accuracy: 0.9337
Epoch 105/180
Learning rate: 0.010000000000000002
391/391 [==============================] - 25s 64ms/step - loss: 0.44

Epoch 145/180
Learning rate: 0.010000000000000002
391/391 [==============================] - 26s 65ms/step - loss: 0.2633 - accuracy: 0.9651 - val_loss: 0.3616 - val_accuracy: 0.9409
Epoch 146/180
Learning rate: 0.010000000000000002
391/391 [==============================] - 26s 65ms/step - loss: 0.2620 - accuracy: 0.9646 - val_loss: 0.3651 - val_accuracy: 0.9430
Epoch 147/180
Learning rate: 0.010000000000000002
391/391 [==============================] - 26s 67ms/step - loss: 0.2621 - accuracy: 0.9633 - val_loss: 0.3671 - val_accuracy: 0.9399
Epoch 148/180
Learning rate: 0.010000000000000002
391/391 [==============================] - 25s 64ms/step - loss: 0.2601 - accuracy: 0.9634 - val_loss: 0.3612 - val_accuracy: 0.9385
Epoch 149/180
Learning rate: 0.010000000000000002
391/391 [==============================] - 26s 66ms/step - loss: 0.2548 - accuracy: 0.9659 - val_loss: 0.3645 - val_accuracy: 0.9396
Epoch 150/180
Learning rate: 0.010000000000000002
391/391 [==========================

D:\anaconda\envs\acnn\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
